In [14]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [15]:
hdf_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/data')
failed_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/failed_queries.csv')
context_path = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/context_data.csv')

# Analyze data aquisition

In [16]:
df_context = pd.read_csv(context_path)
df_context

,circuit_type,circuit_name,timestamp_fgc,STATUS.I_MEAS,IAB.I_A,IEARTH.IEARTH,STATUS.I_EARTH_PCNT,STATUS.I_REF,RR17.RB.A12:U_DUMP_RES,RR17.RB.A12:T_RES_BODY_1,...,B28R1:U_HDS_3,B28R1:U_HDS_4,A28R1:I_HDS_1,A28R1:I_HDS_2,A28R1:I_HDS_3,A28R1:I_HDS_4,A28R1:U_HDS_1,A28R1:U_HDS_2,A28R1:U_HDS_3,A28R1:U_HDS_4
0,RB,RB.A12,1418330714580000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RB,RB.A12,1418384764500000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RB,RB.A12,1418484670560000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,997.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RB,RB.A12,1418538814280000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RB,RB.A12,1418560853860000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,RB,RB.A12,1418593749920000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,997.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RB,RB.A12,1418626484100000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RB,RB.A12,1421531813780000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,RB,RB.A12,1421580539680000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,RB,RB.A12,1446531949860000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_failed = pd.read_csv(failed_dir)
df_failed

,circuit_type,circuit_name,timestamp_fgc,EE_U_DUMP_RES_PM,EE_T_RES_PM,CURRENT_VOLTAGE_DIODE_LEADS_PM,error,LEADS
0,RB,RB.A12,1418330714580000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,NaN,NaN
1,RB,RB.A12,1418384764500000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,NaN,NaN
2,RB,RB.A12,1418484670560000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,Unable to create group (name already exists),empty DataFrame returned
3,RB,RB.A12,1418538814280000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,Unable to create group (name already exists),NaN
4,RB,RB.A12,1418560853860000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,Unable to create group (name already exists),NaN
5,RB,RB.A12,1418593749920000000,empty DataFrame returned,empty DataFrame returned,no DataFrame returned,Unable to create group (name already exists),empty DataFrame returned
6,RB,RB.A12,1418626484100000000,empty DataFrame returned,empty DataFrame returned,NaN,NaN,NaN


# Open h5 files

In [13]:
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/data')

for index, row in df_context.iterrows():
    filename = f"{row['circuit_type']}_{row['circuit_name']}_{int(row['timestamp_fgc'])}.hdf5"
    
    with h5py.File(file_dir / filename, "r") as f:
        print(f.keys())
        print(np.array(f["STATUS.I_MEAS"].get("values")))
        
        
    break

<KeysViewHDF5 ['C17R1:I_HDS_1', 'C17R1:I_HDS_2', 'C17R1:I_HDS_3', 'C17R1:I_HDS_4', 'C17R1:ST_MAGNET_OK', 'C17R1:ST_NQD0', 'C17R1:U_1', 'C17R1:U_2', 'C17R1:U_HDS_1', 'C17R1:U_HDS_2', 'C17R1:U_HDS_3', 'C17R1:U_HDS_4', 'C17R1:U_QS0', 'DFLAS.7L2.RB.A12.LD1:U_HTS', 'DFLAS.7L2.RB.A12.LD1:U_RES', 'DFLAS.7L2.RB.A12.LD2:U_HTS', 'DFLAS.7L2.RB.A12.LD2:U_RES', 'DFLAS.7R1.RB.A12.LD3:U_HTS', 'DFLAS.7R1.RB.A12.LD3:U_RES', 'DFLAS.7R1.RB.A12.LD4:U_HTS', 'DFLAS.7R1.RB.A12.LD4:U_RES', 'IAB.I_A', 'IEARTH.IEARTH', 'I_MEAS_LEADS', 'MB.A10L2:U_DIODE_RB', 'MB.A10L2:U_EARTH_RB', 'MB.A10R1:U_DIODE_RB', 'MB.A10R1:U_EARTH_RB', 'MB.A11L2:U_DIODE_RB', 'MB.A11R1:U_DIODE_RB', 'MB.A12L2:U_DIODE_RB', 'MB.A12R1:U_DIODE_RB', 'MB.A13L2:U_DIODE_RB', 'MB.A13R1:U_DIODE_RB', 'MB.A14L2:U_DIODE_RB', 'MB.A14R1:U_DIODE_RB', 'MB.A15L2:U_DIODE_RB', 'MB.A15R1:U_DIODE_RB', 'MB.A16L2:U_DIODE_RB', 'MB.A16R1:U_DIODE_RB', 'MB.A17L2:U_DIODE_RB', 'MB.A17R1:U_DIODE_RB', 'MB.A18L2:U_DIODE_RB', 'MB.A18R1:U_DIODE_RB', 'MB.A19L2:U_DIODE_RB', 'M

In [ ]:
ds